In [1]:
import sys, os
sys.path.append(os.path.abspath('../..'))

In [2]:
from tqdm.notebook import tqdm
import math
import gym
import torch
import torch.optim as optim 
from torch.utils.tensorboard import SummaryWriter
from collections import deque

from networks.dqn_atari import MC_DQN
from utils.memory import RankedReplayMemory
from utils.optimization import AMN_perc_optimization
from environments.atari_wrappers import make_atari, wrap_deepmind
from utils.atari_utils import fp, ActionSelector, evaluate
from utils.acquisition_functions import mc_var_ratio

In [3]:
env_name = 'Breakout'
env_raw = make_atari('{}NoFrameskip-v4'.format(env_name))
env = wrap_deepmind(env_raw, frame_stack=False, episode_life=False, clip_rewards=True)
c,h,w = c,h,w = fp(env.reset()).shape
n_actions = env.action_space.n

In [4]:
BATCH_SIZE = 32
LR = 0.0000625
GAMMA = 0.99
EPS_START = 1.
EPS_END = 0.1
EPS_DECAY = 1000000
TARGET_UPDATE = 10000
NUM_STEPS = 30000000
M_SIZE = 100000
POLICY_UPDATE = 10000
EVALUATE_FREQ = 100000
SAVE_FREQ = 100000

NAME = 'train_AMN_var_ratio'
FEATURE_REGRESSION = False
PERCENTAGE = 0.2

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # if gpu is to be used
AMN_net = MC_DQN(n_actions).to(device)
expert_net = torch.load("models/dqn_expert_breakout_model").to(device)
AMN_net.apply(AMN_net.init_weights)
expert_net.eval()
optimizer = optim.Adam(AMN_net.parameters(), lr=LR, eps=1.5e-4)

In [6]:
memory = RankedReplayMemory(M_SIZE, [5,h,w], n_actions, mc_var_ratio, AMN_net, device=device)
action_selector = ActionSelector(EPS_START, EPS_END, AMN_net, EPS_DECAY, n_actions, device)

In [7]:
steps_done = 0
writer = SummaryWriter(f'runs/{NAME}')

In [8]:
q = deque(maxlen=5)
done=True
eps = 0
episode_len = 0

In [ ]:
progressive = tqdm(range(NUM_STEPS), total=NUM_STEPS, ncols=400, leave=False, unit='b')
for step in progressive:
  if done:
    env.reset()
    sum_reward = 0
    episode_len = 0
    img, _, _, _ = env.step(1) # BREAKOUT specific !!!
    for i in range(10): # no-op
      n_frame, _, _, _ = env.step(0)
      n_frame = fp(n_frame)
      q.append(n_frame)
        
  train = len(memory) > 50000
  # Select and perform an action
  state = torch.cat(list(q))[1:].unsqueeze(0)
  action, eps = action_selector.select_action(state, training=train)
  n_frame, reward, done, info = env.step(action)
  n_frame = fp(n_frame)

  # 5 frame as memory
  q.append(n_frame)
  memory.push(torch.cat(list(q)).unsqueeze(0), action, reward, done) # here the n_frame means next frame from the previous time step
  episode_len += 1

  # Perform one step of the optimization (on the target network)
  if step % POLICY_UPDATE == 0:
    loss = AMN_perc_optimization(AMN_net, expert_net, optimizer, memory, feature_regression=FEATURE_REGRESSION,
                                 training=train, batch_size=BATCH_SIZE, percentage=PERCENTAGE, device=device)
    if loss is not None:
      writer.add_scalar('Performance/loss', loss, step)
    
    
  if step % EVALUATE_FREQ == 0:
    evaluated_reward = evaluate(step, AMN_net, device, env_raw, n_actions, eps=0.05, num_episode=15)
    writer.add_scalar('Performance/reward', evaluated_reward, step)
    
  if step % SAVE_FREQ == 0:
    torch.save(AMN_net, "models/dqn_AMN_breakout_var_ratio_model")

In [ ]:
torch.save(AMN_net, "models/dqn_AMN_breakout_var_ratio_model")

In [ ]:
AMN_net = torch.load("models/dqn_AMN_breakout_var_ratio_model")